In [2]:
import numpy as np
from hmmlearn import hmm
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import itertools

In [3]:
data = pd.read_csv("Discrete_Fixed.csv")

In [4]:
data.head()

,kmer_1,kmer_2,kmer_3,kmer_4,kmer_5,kmer_6,kmer_7,kmer_8,kmer_9,kmer_10,...,kmer_9990,kmer_9991,kmer_9992,kmer_9993,kmer_9994,kmer_9995,kmer_9996,kmer_9997,kmer_9998,kmer_9999
0,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn,...,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn
1,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn,...,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn
2,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn,...,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn
3,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn,...,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn
4,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn,...,nn,nn,nn,nn,nn,nn,nn,nn,nn,nn


In [5]:
#type(data)
X = []

In [6]:
for i, row in data.iterrows():
    newrow = pd.Series.to_numpy(row)
    X.append(newrow)
# this chunk converts the data frame into a list of numpy arrays for hmmleanr

In [7]:
type(X[100])

numpy.ndarray

In [8]:
#for item in X:
    #print(type(item))

In [9]:
converted = [np.array(i).tolist() for i in X]


In [10]:
type(converted[100])

list

In [11]:
# Generate all 16 possible dinucleotides
bases = ['a', 'c', 'g', 't', 'n']
all_possible_dinucleotides = [''.join(p) for p in itertools.product(bases, repeat=2)]

# Sort for consistent label encoding
unique_dinucleotides = sorted(all_possible_dinucleotides)

label_encoder = LabelEncoder()
label_encoder.fit(unique_dinucleotides)

print("All 16 dinucleotides:", unique_dinucleotides)
print("Mapping learned by LabelEncoder:")
for k, v in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{k} → {v}")

All 16 dinucleotides: ['aa', 'ac', 'ag', 'an', 'at', 'ca', 'cc', 'cg', 'cn', 'ct', 'ga', 'gc', 'gg', 'gn', 'gt', 'na', 'nc', 'ng', 'nn', 'nt', 'ta', 'tc', 'tg', 'tn', 'tt']
Mapping learned by LabelEncoder:
aa → 0
ac → 1
ag → 2
an → 3
at → 4
ca → 5
cc → 6
cg → 7
cn → 8
ct → 9
ga → 10
gc → 11
gg → 12
gn → 13
gt → 14
na → 15
nc → 16
ng → 17
nn → 18
nt → 19
ta → 20
tc → 21
tg → 22
tn → 23
tt → 24


In [12]:
numerical_sequences = []
sequence_lengths = []

for seq in converted:
    numerical_sequence = label_encoder.transform(seq)
    numerical_sequences.append(numerical_sequence)
    sequence_lengths.append(len(numerical_sequence))

print("Example numerical sequence for the first window:", numerical_sequences[555])
print("Lengths of the sequences:", sequence_lengths[:5]) # Print first few lengths
# this chunk converts the list 'converted' into 

Example numerical sequence for the first window: [10  2 10 ... 12 10  2]
Lengths of the sequences: [9999, 9999, 9999, 9999, 9999]


In [13]:
all_sequences = np.concatenate(numerical_sequences).reshape(-1, 1)
sequence_lengths_np = np.array(sequence_lengths) # Convert to NumPy array (optional but good practice)

print("Shape of all_sequences:", all_sequences.shape)
print("Shape of sequence_lengths:", sequence_lengths_np.shape)

Shape of all_sequences: (46695330, 1)
Shape of sequence_lengths: (4670,)


In [19]:
n_components = 2 
n_features = len(unique_dinucleotides) # Number of unique dinucleotides


In [21]:
model = hmm.MultinomialHMM(n_components=n_components,
                            n_trials=len(data[0]), # Each observation is a single dinucleotide
                            n_iter=10000, # Number of EM iterations
                            random_state=42,
                            init_params='ste') # Initialize all parameters

model.fit(all_sequences, lengths=sequence_lengths_np)

print("\nModel trained!")
print("Transition probabilities:\n", model.transmat_)
print("Emission probabilities:\n", model.emissionprob_)
print("Start probabilities:\n", model.startprob_)

KeyError: 0

In [15]:
print(all_sequences.dtype)

int64


In [90]:
probability_data = data.values 
print(probability_data.shape)
n_symbols = probability_data.shape[1]
n_samples = probability_data.shape[0]

(4670, 9999)


In [42]:
probability_data[777]

array(['tc', 'ct', 'ta', ..., 'gc', 'ct', 'tg'],
      shape=(9999,), dtype=object)

In [43]:
valid_rows = probability_data.sum(axis=1) > 0
print(valid_rows.shape)
probability_data = probability_data[valid_rows]
print(probability_data.shape)
probability_data[:10] #no function like head so just slice the first 10 elements

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
samples = np.array([np.random.choice(n_symbols, p=row) for row in probability_data])

observations = samples.reshape(-1, 1)

n_states = 2  
model = hmm.MultinomialHMM(n_components=n_states, n_iter=10000, tol=0.01)

model.fit(observations)

hidden_states = model.predict(observations)

print("Predicted Hidden States:")
print(hidden_states)

print("\nLearned Transition Matrix:")
print(model.transmat_)

print("\nLearned Emission Matrix:")
print(model.emissionprob_)

In [54]:
print(observations.shape)  # Should be (n_samples, 1)


(4037, 1)


In [20]:


# Assuming your DataFrame is named 'data'
print(data.head())
print(data.columns)

# 1. Get all unique dinucleotides from all columns and fit the LabelEncoder
all_dinucleotides = data.values.flatten()
unique_dinucleotides = [x for x in np.unique(all_dinucleotides) if pd.notna(x)] # Exclude NaNs
label_encoder = LabelEncoder()
label_encoder.fit(unique_dinucleotides)

# 2. Convert each row (window) into a NumPy array of numerical indices
train_sequences = []
sequence_lengths = []

for index, row in data.iterrows():
    dinucleotide_sequence = row.values.astype(str) # Get row values as strings
    valid_dinucleotides = [dino for dino in dinucleotide_sequence if pd.notna(dino)] # Exclude NaNs
    numerical_sequence = label_encoder.transform(valid_dinucleotides)
    train_sequences.append(numerical_sequence)
    sequence_lengths.append(len(numerical_sequence))

print("Example training sequence (numerical):", train_sequences[0])
print("Example sequence length:", sequence_lengths[0])
print("Number of training sequences (windows):", len(train_sequences))

  kmer_1 kmer_2 kmer_3 kmer_4 kmer_5 kmer_6 kmer_7 kmer_8 kmer_9 kmer_10  ...  \
0     nn     nn     nn     nn     nn     nn     nn     nn     nn      nn  ...   
1     nn     nn     nn     nn     nn     nn     nn     nn     nn      nn  ...   
2     nn     nn     nn     nn     nn     nn     nn     nn     nn      nn  ...   
3     nn     nn     nn     nn     nn     nn     nn     nn     nn      nn  ...   
4     nn     nn     nn     nn     nn     nn     nn     nn     nn      nn  ...   

  kmer_9990 kmer_9991 kmer_9992 kmer_9993 kmer_9994 kmer_9995 kmer_9996  \
0        nn        nn        nn        nn        nn        nn        nn   
1        nn        nn        nn        nn        nn        nn        nn   
2        nn        nn        nn        nn        nn        nn        nn   
3        nn        nn        nn        nn        nn        nn        nn   
4        nn        nn        nn        nn        nn        nn        nn   

  kmer_9997 kmer_9998 kmer_9999  
0        nn        nn       